In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'lambda': 10,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:1' if torch.cuda.is_available() else 'cpu'), }

#     trial_id = str(uuid.uuid4())
    trial_id = f"seed={params['seed']}_epoch={params['epochs_per_task']}q__"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/seed=1_epoch=1q__


In [2]:
from datasets import MNIST
from datasets import FashionMNIST

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = min(params['per_task_examples'], 10000),
                      random_class_idx = True)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = min(params['per_task_examples'], 15000),
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = cl.benchmarks.SplitCIFAR10(num_tasks=params['num_tasks'],
                                        per_task_memory_examples=params['per_task_memory_examples'])
    n_feature = 32*32*3

params['alpha'] = 0.01
params['lambda'] = 10

[2 9 6 4 0 3 1 7 8 5]


In [3]:
benchmark.trains[1].inputs.shape

torch.Size([11907, 1, 28, 28])

In [4]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [5]:
from algorithms import Weighting
from metrics import MetricCollector2

backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Weighting(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [6]:
from trainers import ContinualTrainer2

trainer = ContinualTrainer2(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())



---------------------------- Task 1 -----------------------
sample_weight.to(device)=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:1')
[1] Eval metrics for task 1 >> {'accuracy': 97.64821166095051, 'loss': 0.00030053448386486694, 'std': 0.07149799094967046}
training_task_end
load_memory_joint: len(train_loader.dataset)=20
---------------------------- Task 2 -----------------------
### Cplex LS solver ###
A.shape=(4, 10000)
b.shape=(4,)
elapsed time:1338.568
[-34.85635341 -33.61251797  33.03582417  35.43305622]
np_weight=tensor([0.1742, 0.1637, 0.1527,  ..., 0.1513, 0.1550, 0.1480])
np_weight.shape=torch.Size([10000])
np_weight.sum()=tensor(1517.7588)
np_weight.max()=tensor(0.1842)
np_weight.min()=tensor(0.0525)
elapsed time:0.011
[-34

In [7]:
metric_manager_callback.meters['std'].get_std()

[0.07149799094967046,
 29.426799818638276,
 19.92448578489145,
 18.57473157276602,
 15.547039495059382]

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[97.648,  0.   ,  0.   ,  0.   ,  0.   ],
       [93.582, 44.021,  0.   ,  0.   ,  0.   ],
       [73.787, 92.887, 44.623,  0.   ,  0.   ],
       [63.449, 87.113, 89.045, 45.816,  0.   ],
       [49.878, 78.247, 82.111, 91.401, 64.202]])

In [9]:
metric_manager_callback.meters['accuracy'].compute_overall()

76.28104193406982